# **MLProcess - Air Quality**
---
**3 - Data Preprocessing**

In [1]:
# Import the required libraries.
import yaml
import joblib
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

## **1 - Configuration File**
---

In [2]:
# Function to load configuration parameter.
def load_config(path_config):
    """
    Load the configuration file (config.yaml).

    Parameters:
    ----------
    path_config : str
        Configuration file location.

    Returns:
    -------
    params : dict
        The configuration parameters.
    """

    # Try to load config.yaml file.
    try:
        with open(path_config, 'r') as file:
            params = yaml.safe_load(file)
    except FileNotFoundError as err:
        raise RuntimeError(f"Configuration file not found in {path_config}")

    return params

In [3]:
# Function to update configuration parameter.
def update_config(key, value, params, path_config):
    """
    Update the configuration parameter values.

    Parameters:
    ----------
    key : str
        The key to be updated.

    value : any type supported in Python
        The updated value.

    params : dict
        Loaded configuration parameters.

    path_config : str
        Configuration file location.

    Returns:
    -------
    config : dict
        Updated configuration parameters.
    """

    # To maintain the raw config immutable.
    params = params.copy()

    # Update the configuration parameters.
    params[key] = value

    with open(path_config, 'w') as file:
        yaml.dump(params, file)

    print(f"Params Updated! \nKey: {key} \nValue: {value}\n")

    # Reload the updated configuration parameters.
    config = load_config(path_config)

    return config

In [4]:
# Load the configuration file.
PATH_CONFIG = "../config/config.yaml"
config = load_config(PATH_CONFIG)

In [5]:
# Check the configuration parameters.
config

{'datetime_columns': ['tanggal'],
 'features': ['stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2'],
 'int32_columns': ['pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max'],
 'label': 'category',
 'label_categories': ['BAIK', 'SEDANG', 'TIDAK SEHAT'],
 'label_categories_new': ['BAIK', 'TIDAK BAIK'],
 'object_columns': ['stasiun', 'critical', 'category'],
 'path_joined_data': '../data/interim/joined_dataset.pkl',
 'path_raw_data': '../data/raw/',
 'path_test_set': ['../data/interim/X_test.pkl', '../data/interim/y_test.pkl'],
 'path_train_set': ['../data/interim/X_train.pkl',
  '../data/interim/y_train.pkl'],
 'path_valid_set': ['../data/interim/X_valid.pkl',
  '../data/interim/y_valid.pkl'],
 'path_validated_data': '../data/interim/validated_data.pkl',
 'range_co': [-1, 47],
 'range_no2': [-1, 65],
 'range_o3': [-1, 151],
 'range_pm10': [-1, 179],
 'range_pm25': [-1, 174],
 'range_so2': [-1, 82],
 'range_stasiun': ['DKI1 (Bunderan HI)',
  'DKI2 (Kelapa Gading)',
  'DKI3 (Jagakarsa)',
  'DK

## **2 - Load Data**
---

In [6]:
# Function to load data.
def load_data(config):
    """
    Load every set of data.

    Parameters:
    ----------
    config : dict
        The loaded configuration file.

    Returns:
    -------
    data_train, data_valid, data_test : pd.DataFrame
        The loaded data.
    """

    # Load the train set.
    X_train = joblib.load(config["path_train_set"][0])
    y_train = joblib.load(config["path_train_set"][1])

    # Load the valid set.
    X_valid = joblib.load(config["path_valid_set"][0])
    y_valid = joblib.load(config["path_valid_set"][1])

    # Load the test set.
    X_test = joblib.load(config["path_test_set"][0])
    y_test = joblib.load(config["path_test_set"][1])

    # Concatenate the X and y of each set.
    data_train = pd.concat([X_train, y_train], axis=1)
    data_valid = pd.concat([X_valid, y_valid], axis=1)
    data_test = pd.concat([X_test, y_test], axis=1)

    # Validate the proportion.
    num_all_data = int(data_train.shape[0]) + int(data_valid.shape[0]) + int(data_test.shape[0])
    print(f"Data train proportion : {len(X_train) / num_all_data}")
    print(f"Data valid proportion : {len(X_valid) / num_all_data}")
    print(f"Data test proportion  : {len(X_test) / num_all_data}")

    return data_train, data_valid, data_test

In [7]:
# Load the data.
data_train, data_valid, data_test = load_data(config)

Data train proportion : 0.7997793712079426
Data valid proportion : 0.09983452840595698
Data test proportion  : 0.10038610038610038


## **3 - Join Categories**
---
`SEDANG` + `TIDAK SEHAT` => `TIDAK BAIK`

In [8]:
# Function to join categories.
def join_categories(set_data, config):
    # Check if label found in set data.
    if config["label"] in set_data.columns.to_list():
        set_data = set_data.copy()

        # Rename SEDANG to TIDAK SEHAT.
        set_data["category"] = set_data["category"].replace("SEDANG", "TIDAK SEHAT")

        # Rename TIDAK SEHAT to TIDAK BAIK.
        set_data["category"] = set_data["category"].replace("TIDAK SEHAT", "TIDAK BAIK")

        return set_data
    else:
        raise RuntimeError("Label is not detected in the dataset.")

In [9]:
# Update the configuration parameter.
config = update_config(
    key = "label_categories_new",
    value = ["BAIK", "TIDAK BAIK"],
    params = config,
    path_config = PATH_CONFIG
)

Params Updated! 
Key: label_categories_new 
Value: ['BAIK', 'TIDAK BAIK']



Join categories in train data.

In [10]:
data_train["category"].value_counts()

category
SEDANG         1044
TIDAK SEHAT     255
BAIK            151
Name: count, dtype: int64

In [11]:
data_train = join_categories(data_train, config)

In [12]:
data_train["category"].value_counts()

category
TIDAK BAIK    1299
BAIK           151
Name: count, dtype: int64

Join categories in valid data.

In [13]:
data_valid["category"].value_counts()

category
SEDANG         130
TIDAK SEHAT     32
BAIK            19
Name: count, dtype: int64

In [14]:
data_valid = join_categories(data_valid, config)

In [15]:
data_valid["category"].value_counts()

category
TIDAK BAIK    162
BAIK           19
Name: count, dtype: int64

Join categories in test data.

In [16]:
data_test["category"].value_counts()

category
SEDANG         131
TIDAK SEHAT     32
BAIK            19
Name: count, dtype: int64

In [17]:
data_test = join_categories(data_test, config)

In [18]:
data_test["category"].value_counts()

category
TIDAK BAIK    163
BAIK           19
Name: count, dtype: int64

## **4 - Handling Missing Value**
---
1. Convert all `-1` values into `NaN`.
2. Impute the missing values for all columns.

In [19]:
# Function to replace -1 with NaN.
def nan_replace(set_data):
    set_data = set_data.copy()
    set_data = set_data.replace(-1, np.nan)
    return set_data

In [20]:
data_train = nan_replace(data_train)
data_train.isnull().sum()

stasiun      0
pm10        42
pm25        69
so2         76
co          12
o3          38
no2         14
category     0
dtype: int64

In [21]:
data_valid = nan_replace(data_valid)
data_valid.isnull().sum()

stasiun      0
pm10         6
pm25        12
so2         10
co           1
o3           5
no2          4
category     0
dtype: int64

In [22]:
data_test = nan_replace(data_test)
data_test.isnull().sum()

stasiun      0
pm10         5
pm25         5
so2         11
co           3
o3           5
no2          1
category     0
dtype: int64

### **4.1. pm10 Imputation**

In [23]:
# pm10 imputed using class mean.
impute_baik = float(data_train[data_train['category'] == 'BAIK']['pm10'].mean())
impute_tidak_baik = float(data_train[data_train['category'] == 'TIDAK BAIK']['pm10'].mean())

print(f"Mean pm10 class BAIK       : {impute_baik}")
print(f"Mean pm10 class TIDAK BAIK : {impute_tidak_baik}")

Mean pm10 class BAIK       : 28.944444444444443
Mean pm10 class TIDAK BAIK : 55.61550632911393


In [24]:
# Update the configuration parameter.
config = update_config(
    key = "impute_pm10",
    value = {"BAIK": impute_baik,
             "TIDAK BAIK": impute_tidak_baik},
    params = config,
    path_config = PATH_CONFIG
)

Params Updated! 
Key: impute_pm10 
Value: {'BAIK': 28.944444444444443, 'TIDAK BAIK': 55.61550632911393}



In [25]:
# Impute the missing values.
data_train.loc[data_train[(data_train.category == 'BAIK') & data_train.pm10.isnull() == True].index, 'pm10'] = impute_baik

data_train.loc[data_train[(data_train.category == 'TIDAK BAIK') & data_train.pm10.isnull() == True].index, 'pm10'] = impute_tidak_baik

print(f"Num of missing value pm10 class BAIK       : {data_train[data_train['category'] == 'BAIK']['pm10'].isnull().sum()}")
print(f"Num of missing value pm10 class TIDAK BAIK : {data_train[data_train['category'] == 'TIDAK BAIK']['pm10'].isnull().sum()}")

Num of missing value pm10 class BAIK       : 0
Num of missing value pm10 class TIDAK BAIK : 0


In [26]:
# Impute the missing values.
data_valid.loc[data_valid[(data_valid.category == 'BAIK') & data_valid.pm10.isnull() == True].index, 'pm10'] = impute_baik

data_valid.loc[data_valid[(data_valid.category == 'TIDAK BAIK') & data_valid.pm10.isnull() == True].index, 'pm10'] = impute_tidak_baik

print(f"Num of missing value pm10 class BAIK       : {data_valid[data_valid['category'] == 'BAIK']['pm10'].isnull().sum()}")
print(f"Num of missing value pm10 class TIDAK BAIK : {data_valid[data_valid['category'] == 'TIDAK BAIK']['pm10'].isnull().sum()}")

Num of missing value pm10 class BAIK       : 0
Num of missing value pm10 class TIDAK BAIK : 0


In [27]:
# Impute the missing values.
data_test.loc[data_test[(data_test.category == 'BAIK') & data_test.pm10.isnull() == True].index, 'pm10'] = impute_baik

data_test.loc[data_test[(data_test.category == 'TIDAK BAIK') & data_test.pm10.isnull() == True].index, 'pm10'] = impute_tidak_baik

print(f"Num of missing value pm10 class BAIK       : {data_test[data_test['category'] == 'BAIK']['pm10'].isnull().sum()}")
print(f"Num of missing value pm10 class TIDAK BAIK : {data_test[data_test['category'] == 'TIDAK BAIK']['pm10'].isnull().sum()}")

Num of missing value pm10 class BAIK       : 0
Num of missing value pm10 class TIDAK BAIK : 0


### **4.2. pm25 Imputation**

In [28]:
# pm25 imputed using class mean.
impute_baik = float(data_train[data_train['category'] == 'BAIK']['pm25'].mean())
impute_tidak_baik = float(data_train[data_train['category'] == 'TIDAK BAIK']['pm25'].mean())

print(f"Mean pm25 class BAIK       : {impute_baik}")
print(f"Mean pm25 class TIDAK BAIK : {impute_tidak_baik}")

Mean pm25 class BAIK       : 39.559633027522935
Mean pm25 class TIDAK BAIK : 82.7256289308176


In [29]:
# Update the configuration parameter.
config = update_config(
    key = "impute_pm25",
    value = {"BAIK": impute_baik,
             "TIDAK BAIK": impute_tidak_baik},
    params = config,
    path_config = PATH_CONFIG
)

Params Updated! 
Key: impute_pm25 
Value: {'BAIK': 39.559633027522935, 'TIDAK BAIK': 82.7256289308176}



In [30]:
# Impute the missing values.
data_train.loc[data_train[(data_train.category == 'BAIK') & data_train.pm25.isnull() == True].index, 'pm25'] = impute_baik

data_train.loc[data_train[(data_train.category == 'TIDAK BAIK') & data_train.pm25.isnull() == True].index, 'pm25'] = impute_tidak_baik

print(f"Num of missing value pm25 class BAIK       : {data_train[data_train['category'] == 'BAIK']['pm25'].isnull().sum()}")
print(f"Num of missing value pm25 class TIDAK BAIK : {data_train[data_train['category'] == 'TIDAK BAIK']['pm25'].isnull().sum()}")

Num of missing value pm25 class BAIK       : 0
Num of missing value pm25 class TIDAK BAIK : 0


In [31]:
# Impute the missing values.
data_valid.loc[data_valid[(data_valid.category == 'BAIK') & data_valid.pm25.isnull() == True].index, 'pm25'] = impute_baik

data_valid.loc[data_valid[(data_valid.category == 'TIDAK BAIK') & data_valid.pm25.isnull() == True].index, 'pm25'] = impute_tidak_baik

print(f"Num of missing value pm25 class BAIK       : {data_valid[data_valid['category'] == 'BAIK']['pm25'].isnull().sum()}")
print(f"Num of missing value pm25 class TIDAK BAIK : {data_valid[data_valid['category'] == 'TIDAK BAIK']['pm25'].isnull().sum()}")

Num of missing value pm25 class BAIK       : 0
Num of missing value pm25 class TIDAK BAIK : 0


In [32]:
# Impute the missing values.
data_test.loc[data_test[(data_test.category == 'BAIK') & data_test.pm25.isnull() == True].index, 'pm25'] = impute_baik

data_test.loc[data_test[(data_test.category == 'TIDAK BAIK') & data_test.pm25.isnull() == True].index, 'pm25'] = impute_tidak_baik

print(f"Num of missing value pm25 class BAIK       : {data_test[data_test['category'] == 'BAIK']['pm25'].isnull().sum()}")
print(f"Num of missing value pm25 class TIDAK BAIK : {data_test[data_test['category'] == 'TIDAK BAIK']['pm25'].isnull().sum()}")

Num of missing value pm25 class BAIK       : 0
Num of missing value pm25 class TIDAK BAIK : 0


### **4.3. so2, co, o3, no2 Imputation**

In [33]:
# so2 imputed using the mean
# co, o3, no2 imputed using the median

impute_so2 = float(data_train["so2"].mean())
impute_co = float(data_train["co"].median())
impute_o3 = float(data_train["o3"].median())
impute_no2 = float(data_train["no2"].median())

impute_values = {
    'so2' : impute_so2,
    'co' : impute_co,
    'o3' : impute_o3,
    'no2' : impute_no2
}

impute_values

{'so2': 35.042940320232894, 'co': 11.0, 'o3': 29.0, 'no2': 18.0}

In [34]:
# Update the configuration parameter.
cols = ['so2', 'co', 'o3', 'no2']
param_keys = ['impute_so2', 'impute_co', 'impute_o3', 'impute_no2']

for col, param_key in zip(cols, param_keys):
    config = update_config(
        key = param_key,
        value = impute_values[col],
        params = config,
        path_config = PATH_CONFIG
    )

Params Updated! 
Key: impute_so2 
Value: 35.042940320232894

Params Updated! 
Key: impute_co 
Value: 11.0

Params Updated! 
Key: impute_o3 
Value: 29.0

Params Updated! 
Key: impute_no2 
Value: 18.0



In [35]:
# Impute the missing values.
data_train = data_train.fillna(value = impute_values)
data_train.isnull().sum()

stasiun     0
pm10        0
pm25        0
so2         0
co          0
o3          0
no2         0
category    0
dtype: int64

In [36]:
data_valid = data_valid.fillna(value = impute_values)
data_valid.isnull().sum()

stasiun     0
pm10        0
pm25        0
so2         0
co          0
o3          0
no2         0
category    0
dtype: int64

In [37]:
data_test = data_test.fillna(value = impute_values)
data_test.isnull().sum()

stasiun     0
pm10        0
pm25        0
so2         0
co          0
o3          0
no2         0
category    0
dtype: int64

In [38]:
data_train["stasiun"].value_counts()

stasiun
DKI1 (Bunderan HI)                  299
DKI5 (Kebon Jeruk) Jakarta Barat    291
DKI3 (Jagakarsa)                    289
DKI4 (Lubang Buaya)                 286
DKI2 (Kelapa Gading)                285
Name: count, dtype: int64

## **5 - Encoding Stasiun**
---

In [39]:
def fit_ohe_encoder(X_stasiun):
    """
    Fit the OHE encoder
    
    Parameters:
    ----------
    X_stasiun : pd.DataFrame
        Categorical input data
    
    Returns:
    -------
    ohe_encoder : sklearn object
        Fitted OHE encoder object
    """
    
    ohe_encoder = OneHotEncoder(sparse_output=False)
    ohe_encoder.fit(np.array(X_stasiun).reshape(-1, 1))
    
    # Serialize the ohe_encoder object.    
    joblib.dump(ohe_encoder, '../models/ohe_stasiun.pkl')
    
    return ohe_encoder

def transform_ohe_encoder(set_data, transformed_column, ohe_path):
    """
    Transform the categorical input data using OHE encoder
    
    Parameters:
    ----------
    set_data : pd.DataFrame
        Data to be transformed.
        
    transformed_column : str
        The column name.
        
    ohe_path : str
        The path to the ohe_encoder object.
        
    Returns:
    -------
    set_data : pd.DataFrame
        The concatenated set data with OHE columns.
    """
    
    set_data = set_data.copy()
    
    # Load the ohe_encoder.
    ohe_encoder = joblib.load(ohe_path)
    
    # Transform the data.
    X_stasiun = np.array(set_data[transformed_column]).reshape(-1, 1)
    stasiun_features = ohe_encoder.transform(X_stasiun)
    
    # Convert to dataframe.    
    stasiun_features = pd.DataFrame(stasiun_features.tolist(), 
                                    columns = list(ohe_encoder.categories_[0]))
    
    # Set index by original set data index.
    stasiun_features.set_index(set_data.index, inplace=True)
    
    # Concatenante the new features with the original set data.
    set_data = pd.concat([stasiun_features, set_data], axis=1)
    
    # Drop the "stasiun" column.
    set_data.drop(columns="stasiun", inplace=True)
    
    # Convert columns type to string.
    new_col = [str(col_name) for col_name in set_data.columns.tolist()]
    set_data.columns = new_col
    
    # Return the feature engineered data.
    return set_data

In [40]:
ohe_stasiun = fit_ohe_encoder(config['range_stasiun'])

In [41]:
# Update the configuration parameter.
PATH_OHE_STASIUN = "../models/ohe_stasiun.pkl"

config = update_config(
    key = "path_ohe_stasiun",
    value = PATH_OHE_STASIUN,
    params = config,
    path_config = PATH_CONFIG
)

Params Updated! 
Key: path_ohe_stasiun 
Value: ../models/ohe_stasiun.pkl



Encode the `stasiun` column in train, valid, and test set.

In [42]:
data_train = transform_ohe_encoder(
    set_data = data_train,
    transformed_column = "stasiun",
    ohe_path = config['path_ohe_stasiun']
)

data_train.head()

,DKI1 (Bunderan HI),DKI2 (Kelapa Gading),DKI3 (Jagakarsa),DKI4 (Lubang Buaya),DKI5 (Kebon Jeruk) Jakarta Barat,pm10,pm25,so2,co,o3,no2,category
235,0.0,0.0,1.0,0.0,0.0,55.615506,86.0,49.0,11.0,29.0,1.0,TIDAK BAIK
1671,0.0,0.0,0.0,0.0,1.0,58.000000,90.0,30.0,19.0,21.0,31.0,TIDAK BAIK
1049,0.0,0.0,0.0,0.0,1.0,54.000000,88.0,31.0,12.0,17.0,28.0,TIDAK BAIK
401,0.0,0.0,1.0,0.0,0.0,46.000000,13.0,16.0,10.0,41.0,4.0,BAIK
1110,0.0,1.0,0.0,0.0,0.0,79.000000,102.0,55.0,14.0,30.0,29.0,TIDAK BAIK


In [43]:
data_valid = transform_ohe_encoder(
    set_data = data_valid,
    transformed_column = "stasiun",
    ohe_path = config['path_ohe_stasiun']
)

data_valid.head()

,DKI1 (Bunderan HI),DKI2 (Kelapa Gading),DKI3 (Jagakarsa),DKI4 (Lubang Buaya),DKI5 (Kebon Jeruk) Jakarta Barat,pm10,pm25,so2,co,o3,no2,category
75,0.0,0.0,1.0,0.0,0.0,69.000000,102.000000,45.00000,14.0,29.0,16.0,TIDAK BAIK
959,0.0,1.0,0.0,0.0,0.0,94.000000,129.000000,51.00000,15.0,50.0,23.0,TIDAK BAIK
548,0.0,0.0,1.0,0.0,0.0,56.000000,82.000000,34.00000,21.0,44.0,24.0,TIDAK BAIK
817,0.0,1.0,0.0,0.0,0.0,55.615506,78.000000,35.04294,16.0,52.0,25.0,TIDAK BAIK
409,0.0,0.0,0.0,1.0,0.0,36.000000,82.725629,30.00000,19.0,55.0,5.0,TIDAK BAIK


In [44]:
data_test = transform_ohe_encoder(
    set_data = data_test,
    transformed_column = "stasiun",
    ohe_path = config['path_ohe_stasiun']
)

data_test.head()

,DKI1 (Bunderan HI),DKI2 (Kelapa Gading),DKI3 (Jagakarsa),DKI4 (Lubang Buaya),DKI5 (Kebon Jeruk) Jakarta Barat,pm10,pm25,so2,co,o3,no2,category
1014,0.0,0.0,1.0,0.0,0.0,76.0,124.0,49.0,18.0,26.0,20.0,TIDAK BAIK
1592,0.0,1.0,0.0,0.0,0.0,75.0,101.0,51.0,15.0,65.0,20.0,TIDAK BAIK
766,0.0,0.0,0.0,0.0,1.0,53.0,85.0,30.0,8.0,27.0,25.0,TIDAK BAIK
954,1.0,0.0,0.0,0.0,0.0,73.0,105.0,29.0,12.0,39.0,35.0,TIDAK BAIK
1068,0.0,0.0,0.0,0.0,1.0,70.0,112.0,32.0,14.0,24.0,21.0,TIDAK BAIK


## **6 - Scaling Data**
---

In [45]:
def fit_scaler(X_concat):
    """
    Fit the scaler
    
    Parameters:
    ----------
    X_concat : pd.DataFrame
        Input data (all features must be in numeric form)
        
    Returns:
    -------
    scaler : sklearn object
        Fitted scaler object (storing the mean & std of all features)
    """
    
    scaler = StandardScaler()
    scaler.fit(X_concat)

    # Serialize the ohe_encoder object.    
    joblib.dump(scaler, '../models/scaler.pkl')
    
    return scaler

def transform_scaler(X_concat, scaler):
    """
    Transform the data using scaler
    
    Parameters:
    ----------
    X_concat : pd.DataFrame
        Input data (all features must be in numeric form)
        
    scaler : sklearn object
        Fitted scaler object (storing the mean & std of all features)
        
    Returns:
    -------
    X_concat_scaled : pd.DataFrame
        Scaled data
    """
    
    X_concat = X_concat.copy()
    
    # Transform the data
    X_concat_scaled = pd.DataFrame(
        scaler.transform(X_concat),
        columns = X_concat.columns,
        index = X_concat.index
    )
    
    return X_concat_scaled

In [46]:
target = data_train['category']

# Fit the scaler
scaler = fit_scaler(data_train.drop(columns='category'))

# Transform the data
data_train = transform_scaler(X_concat = data_train.drop(columns='category'),
                              scaler = scaler)

data_train = pd.concat([data_train, target], axis=1)

data_train.head()

,DKI1 (Bunderan HI),DKI2 (Kelapa Gading),DKI3 (Jagakarsa),DKI4 (Lubang Buaya),DKI5 (Kebon Jeruk) Jakarta Barat,pm10,pm25,so2,co,o3,no2,category
235,-0.50968,-0.494606,2.004321,-0.495686,-0.501077,0.187072,0.314754,1.136291,-0.128454,-0.200923,-2.044531,TIDAK BAIK
1671,-0.50968,-0.494606,-0.498922,-0.495686,1.995700,0.347676,0.476798,-0.410563,1.498256,-0.754800,1.305266,TIDAK BAIK
1049,-0.50968,-0.494606,-0.498922,-0.495686,1.995700,0.078262,0.395776,-0.329149,0.074885,-1.031738,0.970286,TIDAK BAIK
401,-0.50968,-0.494606,2.004321,-0.495686,-0.501077,-0.460566,-2.642550,-1.550350,-0.331793,0.629891,-1.709552,BAIK
1110,-0.50968,2.021811,-0.498922,-0.495686,-0.501077,1.762100,0.962930,1.624771,0.481562,-0.131689,1.081946,TIDAK BAIK


In [47]:
target = data_valid['category']

# Transform the data
data_valid = transform_scaler(X_concat = data_valid.drop(columns='category'),
                              scaler = scaler)

data_valid = pd.concat([data_valid, target], axis=1)

data_valid.head()

,DKI1 (Bunderan HI),DKI2 (Kelapa Gading),DKI3 (Jagakarsa),DKI4 (Lubang Buaya),DKI5 (Kebon Jeruk) Jakarta Barat,pm10,pm25,so2,co,o3,no2,category
75,-0.50968,-0.494606,2.004321,-0.495686,-0.501077,1.088565,0.962930,8.106376e-01,0.481562,-0.200923,-0.369633,TIDAK BAIK
959,-0.50968,2.021811,-0.498922,-0.495686,-0.501077,2.772402,2.056728,1.299118e+00,0.684901,1.253002,0.411987,TIDAK BAIK
548,-0.50968,-0.494606,2.004321,-0.495686,-0.501077,0.212969,0.152710,-8.490926e-02,1.904934,0.837594,0.523646,TIDAK BAIK
817,-0.50968,2.021811,-0.498922,-0.495686,-0.501077,0.187072,-0.009334,-5.784766e-16,0.888240,1.391471,0.635306,TIDAK BAIK
409,-0.50968,-0.494606,-0.498922,2.017407,-0.501077,-1.134101,0.182106,-4.105627e-01,1.498256,1.599174,-1.597892,TIDAK BAIK


In [48]:
target = data_test['category']

# Transform the data
data_test = transform_scaler(X_concat = data_test.drop(columns='category'),
                             scaler = scaler)

data_test = pd.concat([data_test, target], axis=1)

data_test.head()

,DKI1 (Bunderan HI),DKI2 (Kelapa Gading),DKI3 (Jagakarsa),DKI4 (Lubang Buaya),DKI5 (Kebon Jeruk) Jakarta Barat,pm10,pm25,so2,co,o3,no2,category
1014,-0.509680,-0.494606,2.004321,-0.495686,-0.501077,1.560039,1.854173,1.136291,1.294917,-0.408627,0.077007,TIDAK BAIK
1592,-0.509680,2.021811,-0.498922,-0.495686,-0.501077,1.492686,0.922419,1.299118,0.684901,2.291519,0.077007,TIDAK BAIK
766,-0.509680,-0.494606,-0.498922,-0.495686,1.995700,0.010909,0.274243,-0.410563,-0.738470,-0.339392,0.635306,TIDAK BAIK
954,1.962014,-0.494606,-0.498922,-0.495686,-0.501077,1.357979,1.084463,-0.491976,0.074885,0.491422,1.751905,TIDAK BAIK
1068,-0.509680,-0.494606,-0.498922,-0.495686,1.995700,1.155918,1.368040,-0.247736,0.481562,-0.547096,0.188667,TIDAK BAIK


## **7 - Balancing Label**
---

In [49]:
data_train["category"].value_counts()

category
TIDAK BAIK    1299
BAIK           151
Name: count, dtype: int64

In [50]:
# Undersampling.
rus = RandomUnderSampler(random_state = 123)

X_rus, y_rus = rus.fit_resample(data_train.drop('category', axis=1),
                                data_train['category'])

data_train_rus = pd.concat([X_rus, y_rus], axis=1)

In [51]:
# Oversampling.
ros = RandomOverSampler(random_state = 123)

X_ros, y_ros = ros.fit_resample(data_train.drop('category', axis=1),
                                data_train['category'])

data_train_ros = pd.concat([X_ros, y_ros], axis=1)

In [52]:
# SMOTE.
smote = SMOTE(random_state = 123)

X_sm, y_sm = smote.fit_resample(data_train.drop('category', axis=1),
                                data_train['category'])

data_train_sm = pd.concat([X_sm, y_sm], axis=1)

## **8 - Label Encoding**
---

In [53]:
def fit_le_encoder(y_categori):
    """
    Fit the LE encoder
    
    Parameters:
    ----------
    y_categori : pd.Series
        Categorical input label
        
    Returns:
    -------
    le_encoder : sklearn object
        Fitted LE encoder object
    """
    
    le_encoder = LabelEncoder()
    le_encoder.fit(y_categori)

    # Serialize the ohe_encoder object.    
    joblib.dump(le_encoder, '../models/le_encoder.pkl')
    
    return le_encoder

def transform_le_encoder(y_categori, le_encoder):
    """
    Transform the categorical input label using LE encoder
    
    Parameters:
    ----------
    y_categori : pd.Series
        Categorical input label
        
    le_encoder : sklearn object
        Fitted LE encoder object
        
    Returns:
    -------
    y_categori_encoded : pd.DataFrame
        Encoded categorical input label
    """
    
    y_categori = y_categori.copy()
    
    # Transform the data
    y_categori_encoded = pd.Series(
        le_encoder.transform(y_categori),        
    )
    
    return y_categori_encoded

In [54]:
# Fit the label encoder.
le_category = fit_le_encoder(config["label_categories_new"])

In [55]:
# Transform RUS data.
y_rus = transform_le_encoder(y_rus, le_category)

In [56]:
# Transform ROS data.
y_ros = transform_le_encoder(y_ros, le_category)

In [57]:
# Transform SMOTE data.
y_sm = transform_le_encoder(y_sm, le_category)

In [58]:
# Transform valid set.
category_encoded = transform_le_encoder(data_valid["category"], le_category)

data_valid["category"] = category_encoded.values.tolist()

In [59]:
# Transform test set.
category_encoded = transform_le_encoder(data_test["category"], le_category)

data_test["category"] = category_encoded.values.tolist()

## **9 - Data Serialization**
---

In [60]:
# undersampling
joblib.dump(X_rus, "../data/processed/X_rus.pkl")
joblib.dump(y_rus, "../data/processed/y_rus.pkl")

# oversampling
joblib.dump(X_ros, "../data/processed/X_ros.pkl")
joblib.dump(y_ros, "../data/processed/y_ros.pkl")

# smote
joblib.dump(X_sm, "../data/processed/X_sm.pkl")
joblib.dump(y_sm, "../data/processed/y_sm.pkl")

# data valid
joblib.dump(data_valid.drop(columns='category'), "../data/processed/X_valid_feng.pkl")
joblib.dump(data_valid['category'], "../data/processed/y_valid_feng.pkl")

# data test
joblib.dump(data_test.drop(columns='category'), "../data/processed/X_test_feng.pkl")
joblib.dump(data_test['category'], "../data/processed/y_test_feng.pkl")

['../data/processed/y_test_feng.pkl']